In [1]:
## import libraries
import os, sys
import yaml
import xarray as xr
import pandas as pd
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
import dask
from datetime import timedelta
import glob
import itertools
%matplotlib inline

# plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import textwrap

import matplotlib as mpl
mpl.use('agg')

# # import personal modules
# sys.path.append('../modules')
# import ar_funcs

dask.config.set(**{'array.slicing.split_large_chunks': True})

sys.path.append('../../mclimate_tool_cw3e')
from build_html_table import create_html_table
import custom_cmaps as ccmap
# from plotter import draw_basemap, plot_mclimate_forecast
from plotter import draw_basemap

import mclimate_funcs as mclim_func

In [2]:
def plot_mclimate_forecast_four_panel(ds, fc, step, fname, domain):
    if domain == 'SEAK':
        ext = [-141., -130., 54., 60.]

    else:
        ext = [-170., -120., 40., 65.]
        
    ls = ds.isel(lat=0).lat.values
    le = ds.isel(lat=-1).lat.values

    if ls < le:
        ds = ds.sel(lon=slice(ext[0], ext[1]), lat=slice(ext[2], ext[3]))
        fc = fc.sel(lon=slice(ext[0], ext[1]), lat=slice(ext[2], ext[3]))
    else:
        ds = ds.sel(lon=slice(ext[0], ext[1]), lat=slice(ext[3], ext[2]))
        fc = fc.sel(lon=slice(ext[0], ext[1]), lat=slice(ext[3], ext[2]))

    ts = pd.to_datetime(ds.init_date.values, format="%Y%m%d%H") 
    init_date = ts.strftime('%Y%m%d%H')
    init_time = ts.strftime('%HZ %d %b %Y')
    start_date = ts - timedelta(days=45)
    start_date = start_date.strftime('%d-%b')
    end_date = ts + timedelta(days=45)
    end_date = end_date.strftime('%d-%b')
    ts_valid = ts + timedelta(hours=int(step))
    valid_time = ts_valid.strftime('%HZ %d %b %Y')
    left_lbl = 'Initialized: {0}'.format(init_time)
    right_lbl = 'F-{0} | Valid: {1}'.format(int(step), valid_time)
    
    # Set up projection
    mapcrs = ccrs.PlateCarree()
    datacrs = ccrs.PlateCarree()
    
    # Set tick/grid locations
    lats = ds.lat.values
    lons = ds.lon.values
    if domain == 'NPAC':
        dx = [-160, -150, -140, -130]
        dy = [45., 50., 55., 60.]
    elif domain == 'SEAK':
        dx = [-140, -135, -130]
        dy = [54., 56., 58., 60.]
    else:
        dx = np.arange(lons.min().round(),lons.max().round()+10,10)
        dy = np.arange(lats.min().round(),lats.max().round()+10,10)
    
    # Create figure
    fig = plt.figure(figsize=(10, 8.5))
    fig.dpi = 600
    fmt = 'png'
    
    nrows = 7
    ncols = 2
    
    # contour labels
    kw_clabels = {'fontsize': 7, 'inline': True, 'inline_spacing': 7, 'fmt': '%i',
                  'rightside_up': True, 'use_clabeltext': True}
    
    kw_ticklabels = {'size': 10, 'color': 'dimgray', 'weight': 'light'}
    
    ## Use gridspec to set up a plot with a series of subplots that is
    ## n-rows by n-columns
    gs = GridSpec(nrows, ncols, height_ratios=[0.5, 0.05, 0.05, 0.5, 0.05, 0.05, 0.05], width_ratios = [1, 1], wspace=0.05, hspace=0.002)
    ## use gs[rows index, columns index] to access grids


    ## loop through variables
    row_idx = [0, 0, 3]
    col_idx = [0, 1, 0]
    var_lst = ['ivt', 'freezing_level', 'uv']
    llat_lst = [True, False, True]
    for i, (row, col, var) in enumerate(zip(row_idx, col_idx, var_lst)):
        ax = fig.add_subplot(gs[row, col], projection=mapcrs)   
        ax = draw_basemap(ax, extent=ext, xticks=dx, yticks=dy, left_lats=llat_lst[i], right_lats=False, bottom_lons=True)
        
        ## set cmap and contour values based on varname
        if var == 'ivt':
            cmap_name = 'mclimate_green'
            clevs = np.arange(250., 2100., 250.)
        elif var == 'freezing_level':
            cmap_name = 'mclimate_red'
            clevs = np.arange(0., 60000., 2000.)
            fc[var] = fc[var]*3.281 # convert to feet
        elif var == 'uv':
            cmap_name = 'mclimate_purple'
            clevs = np.arange(0., 55., 5.)
        
        # Contour Filled (mclimate values)
        data = ds.sel(step=step)[var].values*100.
        cmap, norm, bnds, cbarticks, cbarlbl = ccmap.cmap(cmap_name)
        cf = ax.pcolormesh(lons, lats, data, transform=datacrs,
                           cmap=cmap, norm=norm, alpha=0.9)

        # Contour Lines (forecast values)
        forecast = fc[var].sel(step=step)     
        cs = ax.contour(lons, lats, forecast, transform=datacrs,
                         levels=clevs, colors='k',
                         linewidths=0.75, linestyles='solid')
        plt.clabel(cs, **kw_clabels)
        
        # Add color bar
        cbax = plt.subplot(gs[row+1,col]) # colorbar axis
        cbarticks = list(itertools.compress(bnds, cbarticks)) ## this labels the cbarticks based on the cmap dictionary
        cb = Colorbar(ax = cbax, mappable = cf, orientation = 'horizontal', 
                      ticklocation = 'bottom', ticks=cbarticks)
        cb.set_label(cbarlbl, fontsize=11)
        cb.ax.tick_params(labelsize=12)

        if i == 0:
            ax.set_title(left_lbl, loc='left', fontsize=10)
        elif i == 1:
            ax.set_title(right_lbl, loc='right', fontsize=10)
    
    txt = 'Relative to all {2}-h GEFSv12 reforecasts initialized between {0} and {1} (2000-2019)'.format(start_date, end_date, step)
    ann_ax = fig.add_subplot(gs[-1, :])
    ann_ax.axis('off')
    ann_ax.annotate(textwrap.fill(txt, 101), # this is the text
               (0, 0.3), # these are the coordinates to position the label
                textcoords="offset points", # how to position the text
                xytext=(0,-19), # distance from text to points (x,y)
                ha='left', # horizontal alignment can be left, right or center
                **kw_ticklabels)
    
    fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)

    plt.close(fig)

In [3]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [4]:
### list of dates currently ran
## '20051120' 'GEFSv12_reforecast'
## '20070607' 'GEFSv12_reforecast'
## '20150813' 'GEFSv12_reforecast'
## '20201128' 'GEFS_archive'
## '20231117' 'GEFS_archive'
## '20231114' 'GEFS_archive'

In [5]:
%%time
######################
### VARS TO UPDATE ###
######################
fdate = '20231117' ## initialization date in YYYYMMDD format
model = 'GEFS_archive' ## 'GEFSv12_reforecast', 'GFS', 'GEFS', 'GEFS_archive'
map_ext = [-170., -120., 40., 65.] ## map extent [minlon, maxlon, minlat, maxlat]
# [-170., -125., 50., 75.] AK
# [-170., -120., 50., 75.] SEAK/NPAC
table_ext = [-141., -130., 54.5, 60.] ## extent to choose the maximum value from for the table [minlon, maxlon, minlat, maxlat]
# [-141., -130., 54.5, 60.] SEAK
# [-160., -145., 65., 70.] Northern AK
fig_path = path_to_figs + 'images_historical/mclimate_{0}/'.format(fdate)
os.makedirs(os.path.dirname(fig_path), exist_ok=True)

####################################
### COMPARE FORECAST TO MCLIMATE ###
####################################
var_lst = ['ivt', 'freezing_level', 'uv1000']
ds_lst = []
fc_lst = []
for i, varname in enumerate(var_lst):
    forecast, ds = mclim_func.run_compare_mclimate_forecast(varname, fdate, model, server='expanse')
    if varname == 'uv1000':
        ds = ds.rename({'mclimate': 'uv'})
    else:
        ds = ds.rename({'mclimate': varname})
    fc_lst.append(forecast)
    ds_lst.append(ds)


### merge the datasets
ds = xr.merge(ds_lst)
ds = ds.sortby('lat')

fc = xr.merge(fc_lst)
fc = fc.sortby('lat')

11 17
11 17
11 17
CPU times: user 3.46 s, sys: 1.21 s, total: 4.67 s
Wall time: 6.3 s


In [6]:
ds

<xarray.Dataset>
Dimensions:         (step: 40, lat: 121, lon: 140)
Coordinates:
  * lat             (lat) float64 10.0 10.5 11.0 11.5 ... 68.5 69.0 69.5 70.0
  * lon             (lon) float64 -179.5 -179.0 -178.5 ... -111.0 -110.5 -110.0
  * step            (step) int64 6 12 18 24 30 36 42 ... 210 216 222 228 234 240
    init_date       datetime64[ns] 2023-11-17
    surface         float64 0.0
    isothermZero    float64 0.0
    isobaricInhPa   float64 1e+03
Data variables:
    ivt             (step, lat, lon) float64 nan nan nan nan ... nan nan nan nan
    freezing_level  (step, lat, lon) float64 0.92 0.9 0.75 0.75 ... nan nan nan
    uv              (step, lat, lon) float64 nan nan nan nan ... nan nan nan nan

In [7]:

step_lst = ds.step.values
for i, step in enumerate(step_lst):
    print(step)
    out_fname = fig_path + 'SEAK_mclimate_F{0}'.format(step)
    plot_mclimate_forecast_four_panel(ds, fc, step, out_fname, domain="SEAK")
    out_fname = fig_path + 'NPAC_mclimate_F{0}'.format(step)
    plot_mclimate_forecast_four_panel(ds, fc, step, out_fname, domain="NPAC")


6
12
18
24
30
36
42
48
54
60
66
72
78
84
90
96
102
108
114
120
126
132
138
144
150
156
162
168
174
180
186
192
198
204
210
216
222
228
234
240


In [8]:
###################
### BUILD TABLE ###
###################
df = create_html_table(ds, domain="SEAK")
df



In [9]:
# ## write to text file
# html = df.to_html(index=False, formatters={'Hour': lambda x: '<b>' + x + '</b>'}, escape=False)

# # write html to file
# text_file = open("../out/table_{0}.html".format(fdate), "w")
# text_file.write(html)
# text_file.close()

In [10]:
#######################
### WRITE HTML FILE ###
#######################
print('...Writing HTML file')
out_fname = "/data/projects/website/mirror/htdocs/Projects/MClimate/mclimate_tool_operational.html"

with open('/data/projects/operations/GEFS_Mclimate/out/html_text.txt', mode='r') as in_file, \
     open('/data/projects/operations/GEFS_Mclimate/out/html_text2.txt', mode='r') as in_file2, \
     open(out_fname, mode='w') as out_file:

    # A file is iterable
    # We can read each line with a simple for loop
    for line in in_file:
        out_file.write(line)
        
        
    ## now add in the table
    out_file.write(df_html)

    ## now add the last few lines
    for line in in_file2:
        out_file.write(line)
        
    out_file.close()

...Writing HTML file


FileNotFoundError: [Errno 2] No such file or directory: '/data/projects/operations/GEFS_Mclimate/out/html_text.txt'